In [5]:
using Pkg
Pkg.activate("")
using JAXTAM

"/home/robert/Projects/JAXTAM/Project.toml"

# Mission Configuration

Julia uses multiple dispatch, where functions can have the same name, but will perform different actions depending on the types of inputs provided. JAXTAM uses this to handle different missions

There is an abstract type called `Mission`, and various functions take in subtypes of `Mission` to perform different actions:

In [7]:
?JAXTAM.Mission

```
Mission
```

Each mission must have some defined functions:

`_mission_name(<:Mission)` - returns the name of the mission

`_mission_master_url(<:Mission)` - returns the url to the HEASARC master table (.tdat.gz) file

`_mission_paths(<:Mission)` - returns a `NamedTuple` with entries corresponding to folder and file paths:

  * download - path the observation folder will be downloaded to from the FTP
  * jaxtam   - path JAXTAM-generated files will be saved to
  * web      - path the web report files will be saved to
  * rmf      - path to the **local** copy of the mission RMF file from caldb

`_mission_path_obs_server(<:Mission, obs_row::Union{DataFrame,DataFrameRow{DataFrame}})` - returns the **server-side** path to an observation

`_mission_good_e_range(<:Mission)` - returns a `Tuple{Float64,Float64}` of the **low** and **high** good energy ranges, respectively

`_mission_instruments(<:Mission)` - returns a `Array{Symbol1,}` with each element as a symbol of the instrument name, e.g. [:FPMA, :FPMB] for NuSTAR

Look at /src/missions/nicer.jl for an example. Put any custom mission functions in /src/missions/custom/ folder.


So, for each mission a new type is created with the mission name, and functions are made which take this type in and return the relevant values

A constant object of this type is then initialised, and exported to the global scope

Check the code in `src/missions/base/nicer.jl` to see how this works

For example, the NICER mission provides:

In [8]:
JAXTAM.NICER # The NICER type

JAXTAM.NICER

In [9]:
nicer # An exported constant object, equivalent to nicer = JAXTAM.NICER()

JAXTAM.NICER()

In [10]:
JAXTAM._mission_name(nicer)

"nicer"

In [11]:
JAXTAM._mission_master_url(nicer)

"https://heasarc.gsfc.nasa.gov/FTP/heasarc/dbase/tdat_files/heasarc_nicermastr.tdat.gz"

In [12]:
JAXTAM._mission_good_e_range(nicer)

(0.2, 12.0)

In [13]:
JAXTAM._mission_instruments(nicer)

1-element Array{Symbol,1}:
 :XTI

Additionally, there is a `mission_paths` function which is used by the user to set up the various directory paths required for download and analysis. This will attempt to read the paths from a json file, and if no entries for the mission exist it will prompt the user for the paths

In [20]:
JAXTAM.mission_paths(nicer)

(web = "/media/robert/data/heasarc/nicer/web", jaxtam = "/media/robert/data/heasarc/nicer/jaxtam", rmf = "/home/robert/Software/caldb/data/nicer/xti/cpf/rmf/nixtiref20170601v001.rmf", download = "/media/robert/data/heasarc/nicer/download")

There's also the very important `_mission_path_obs_server` function, which will be covered later

This multiple-dispatch style is used to make it easy to expand or build on the code base. For example, a user might require some other mission-related function and want to easily integrate this with the JAXTAM module. All that needs to be done is to dispatch that function onto the `NICER` type:

In [17]:
_mission_max_temp(::JAXTAM.NICER) = 200

_mission_max_temp(nicer)

200

# Master Tables

Using the `_mission_master_url` function, JAXTAM can download and set up the mission's master table:

In [21]:
JAXTAM.master(nicer)

┌ Info: Loading /media/robert/data/heasarc/nicer/jaxtam/master.feather
└ @ JAXTAM /home/robert/Projects/JAXTAM/src/io/master_tables.jl:133
┌ Info: Loading /media/robert/data/heasarc/nicer/jaxtam/append.feather
└ @ JAXTAM /home/robert/Projects/JAXTAM/src/io/master_tables.jl:219


,name,ra,dec,lii,bii,time,end_time,obsid,exposure,time_awarded,num_fpm,processing_status,processing_date,public_date,processing_version,num_processed,caldb_version,software_version,prnb,abstract,subject_category,category_code,pi_lname,pi_fname,cycle,obs_type,title,remarks,publicity,logged,downloaded
,String,Float64,Float64,Float64,Float64,Arrow…,Arrow…,String,Float64,Float64,Int64,String,Arrow…,Arrow…,String,Int64,String,String,String,String,String,Int64,String,String,Int64,String,String,String,Bool,Bool,Bool
1,1A_0535+262,84.3791,26.1659,181.405,-2.98776,2018-09-24T14:04:52,2018-09-24T14:32:19,1200360101,0.0,0.0,52,VALIDATED,2018-10-21T16:58:29,2018-10-08T00:00:00,l0-master_20181010,4,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-10-07_V005,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,true,false
2,1A_0535+262,84.7281,26.31,181.45,-2.64605,2018-09-27T22:30:00,2018-09-27T22:47:00,1200360104,901.0,0.0,52,VALIDATED,2018-10-08T00:17:17,2018-10-11T00:00:00,l0-master_20180712,2,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-07-12_V004a,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,false,false
3,1A_0535+262,84.7269,26.3154,181.445,-2.64411,2018-09-26T01:27:59,2018-09-26T06:29:46,1200360103,3830.0,0.0,52,VALIDATED,2018-10-08T00:13:27,2018-10-10T00:00:00,l0-master_20180712,2,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-07-12_V004a,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,false,false
4,1A_0535+262,84.7269,26.3156,181.445,-2.64399,2018-09-30T01:12:19,2018-09-30T01:38:03,1200360105,1429.0,0.0,52,VALIDATED,2018-10-08T00:19:17,2018-10-14T00:00:00,l0-master_20180712,2,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-07-12_V004a,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,false,false
5,1A_0535+262,84.7271,26.3156,181.445,-2.64387,2018-09-25T06:46:44,2018-09-25T07:26:25,1200360102,1321.0,0.0,52,VALIDATED,2018-10-21T18:40:02,2018-10-09T00:00:00,l0-master_20181010,3,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-10-07_V005,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,false,false
6,1E_0102.2-7219,15.9898,-72.064,301.568,-45.0311,2017-06-19T19:01:25,2017-06-19T19:13:02,0010010101,3.0,0.0,52,PROCESSED,2018-03-02T21:41:19,2099-12-31T00:00:00,l0-master_20180226,3,xti20180226,Hea_10Oct2017_V6.22.1_NICER_2018-03-01_V003,0010,Observations of general NICER calibration,CAL,1,GENDREAU,KEITH C.,0,CAL,CALIBRATION,missing,false,false,false
7,1E_0102.2-7219,16.016,-72.0626,301.557,-45.032,2017-06-22T00:42:42,2017-06-22T08:39:55,0010010104,3.0,0.0,52,PROCESSED,2018-03-02T21:50:08,2099-12-31T00:00:00,l0-master_20180226,3,xti20180226,Hea_10Oct2017_V6.22.1_NICER_2018-03-01_V003,0010,Observations of general NICER calibration,CAL,1,GENDREAU,KEITH C.,0,CAL,CALIBRATION,missing,false,false,false
8,1E_0102.2-7219,16.0194,-72.0617,301.555,-45.0328,2017-06-21T01:33:40,2017-06-21T23:44:42,0010010103,11.0,0.0,52,PROCESSED,2018-03-02T21:44:26,2099-12-31T00:00:00,l0-master_20180226,3,xti20180226,Hea_10Oct2017_V6.22.1_NICER_2018-03-01_V003,0010,Observations of general NICER calibration,CAL,1,GENDREAU,KEITH C.,0,CAL,CALIBRATION,missing,false,false,false
9,1E_0102.2-7219,16.0212,-72.0609,301.554,-45.0335,2017-06-20T02:43:03,2017-06-20T19:57:38,0010010102,6.0,0.0,52,PROCESSED,2018-03-02T21:42:35,2099-12-31T00:00:00,l0-master_20180226,3,xti20180226,Hea_10Oct2017_V6.22.1_NICER_2018-03-01_V003,0010,Observations of general NICER calibration,CAL,1,GENDREAU,KEITH C.,0,CAL,CALIBRATION,missing,false,false,false


Query functionality exists to allow for easy selection of observations:

In [22]:
JAXTAM.master_query(nicer, :name, "1A_0535+262")

,name,ra,dec,lii,bii,time,end_time,obsid,exposure,time_awarded,num_fpm,processing_status,processing_date,public_date,processing_version,num_processed,caldb_version,software_version,prnb,abstract,subject_category,category_code,pi_lname,pi_fname,cycle,obs_type,title,remarks,publicity,logged,downloaded
,String,Float64,Float64,Float64,Float64,Arrow…,Arrow…,String,Float64,Float64,Int64,String,Arrow…,Arrow…,String,Int64,String,String,String,String,String,Int64,String,String,Int64,String,String,String,Bool,Bool,Bool
1,1A_0535+262,84.3791,26.1659,181.405,-2.98776,2018-09-24T14:04:52,2018-09-24T14:32:19,1200360101,0.0,0.0,52,VALIDATED,2018-10-21T16:58:29,2018-10-08T00:00:00,l0-master_20181010,4,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-10-07_V005,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,true,false
2,1A_0535+262,84.7281,26.31,181.45,-2.64605,2018-09-27T22:30:00,2018-09-27T22:47:00,1200360104,901.0,0.0,52,VALIDATED,2018-10-08T00:17:17,2018-10-11T00:00:00,l0-master_20180712,2,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-07-12_V004a,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,false,false
3,1A_0535+262,84.7269,26.3154,181.445,-2.64411,2018-09-26T01:27:59,2018-09-26T06:29:46,1200360103,3830.0,0.0,52,VALIDATED,2018-10-08T00:13:27,2018-10-10T00:00:00,l0-master_20180712,2,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-07-12_V004a,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,false,false
4,1A_0535+262,84.7269,26.3156,181.445,-2.64399,2018-09-30T01:12:19,2018-09-30T01:38:03,1200360105,1429.0,0.0,52,VALIDATED,2018-10-08T00:19:17,2018-10-14T00:00:00,l0-master_20180712,2,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-07-12_V004a,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,false,false
5,1A_0535+262,84.7271,26.3156,181.445,-2.64387,2018-09-25T06:46:44,2018-09-25T07:26:25,1200360102,1321.0,0.0,52,VALIDATED,2018-10-21T18:40:02,2018-10-09T00:00:00,l0-master_20181010,3,xti20180711,Hea_24Apr2018_V6.24_NICER_2018-10-07_V005,1200,Target of Opportunities sponsored by the Principal Investigator,DIR,9,GENDREAU,KEITH C.,1,DDT_TOO,Director Target of Operations,missing,true,false,false


In [24]:
obs_row = JAXTAM.master_query(nicer, :obsid, "1200360102")

DataFrameRow (row 1)
name                1A_0535+262
ra                  84.72708
dec                 26.31559
lii                 181.44505501
bii                 -2.64387286
time                2018-09-25T06:46:44
end_time            2018-09-25T07:26:25
obsid               1200360102
exposure            1321.0
time_awarded        0.0
num_fpm             52
processing_status   VALIDATED
processing_date     2018-10-21T18:40:02
public_date         2018-10-09T00:00:00
processing_version  l0-master_20181010
num_processed       3
caldb_version       xti20180711
software_version    Hea_24Apr2018_V6.24_NICER_2018-10-07_V005
prnb                1200
abstract            Target of Opportunities sponsored by the Principal Investigator
subject_category    DIR
category_code       9
pi_lname            GENDREAU
pi_fname            KEITH C.
cycle               1
obs_type            DDT_TOO
title               Director Target of Operations
remarks             missing
publicity           true
logged  

Query will either return a subset of the master `DataFrame` if multiple observations match the query, or an individual `DataFrameRow` if only one observation is returned.

Previously the `_obs_path_server` function was mentioned as being part of the mission configuration. This function takes in a row from the master table, and using the information in that row returns the path of the observation on the HEASARC FTP server:

In [27]:
JAXTAM._obs_path_server(nicer, obs_row)

"/.nicer_archive/.nicer_201809a/obs/2018_09/1200360102"

Using the paths set in the json path config file, a function can also return the local path, depnding on which kind of path is being searched for:

In [30]:
JAXTAM._obs_path_local(nicer, obs_row; kind=:download)

"/media/robert/data/heasarc/nicer/download/nicer_archive/nicer_201809a/obs/2018_09/1200360102"

In [31]:
JAXTAM._obs_path_local(nicer, obs_row; kind=:jaxtam)

"/media/robert/data/heasarc/nicer/jaxtam/nicer_archive/nicer_201809a/obs/2018_09/1200360102"

In [32]:
JAXTAM._obs_path_local(nicer, obs_row; kind=:web)

"/media/robert/data/heasarc/nicer/web/nicer_archive/nicer_201809a/obs/2018_09/1200360102"

Depending on the use case, it may be desireable to have these paths all be in different locations, or all to be identical. For example, splitting the `download` path out from the `jaxtam` and `web` paths can be useful if you want to easily delete the raw downloaded observation once the `jaxtam` analysis files have been generated, or it may be useful to directly place the `web` path in a `public_html` folder

# Downloading Observations

